# Fine-tuning Whisper for Vietnamese ASR
This notebook demonstrates how to fine-tune OpenAI's Whisper model on a Vietnamese speech dataset using Hugging Face Transformers. The workflow includes environment setup, data loading, preprocessing, model training, and evaluation.

## 1. Environment Setup
Install the required libraries: `transformers`, `datasets`, `torchaudio`, and `jiwer` for evaluation.

In [2]:
!pip install transformers datasets torchaudio jiwer --quiet

In [3]:
import os
import pandas as pd
from datasets import load_dataset, Dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, TrainingArguments, Trainer
import torch

## 2. Load and Prepare Data
Assume your CSV files (`fpt_train.csv`, `fpt_val.csv`, `fpt_test.csv`) are in the current directory and contain columns: `path` (audio file path) and `transcription` (text).

In [4]:
def load_csv_to_dataset(csv_path):
    df = pd.read_csv(csv_path)
    ds = Dataset.from_pandas(df)
    ds = ds.cast_column("path", Audio(sampling_rate=16000))
    return ds

train_dataset = load_csv_to_dataset("fpt_train.csv")
val_dataset = load_csv_to_dataset("fpt_val.csv")
test_dataset = load_csv_to_dataset("fpt_test.csv")

## 3. Load Whisper Model and Processor

In [5]:
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="vi", task="transcribe")

## 4. Preprocessing Function
Tokenize transcriptions and prepare input features.

In [ ]:
import gc
from datasets import Dataset

def prepare_single_example(example):
    audio_data = example["path"]
    transcription = example["transcription"]
    features = processor.feature_extractor(
        audio_data["array"], 
        sampling_rate=audio_data["sampling_rate"]
    ).input_features[0]
    label = processor.tokenizer(transcription).input_ids
    return {"input_features": features, "labels": label}

# Gộp cả train, val, test lại thành một list lớn
all_examples = list(train_dataset) + list(val_dataset) + list(test_dataset)

# Xử lý toàn bộ dữ liệu một lượt
processed_all_data = []
for i, example in enumerate(all_examples):
    processed_all_data.append(prepare_single_example(example))
    # Ghi tiến độ vào file
    with open("preprocessing_progress.txt", "w") as f:
        f.write(str(i))
    if (i + 1) % 100 == 0:
        print(f"Đã xử lý {i + 1} mẫu tổng hợp. Giải phóng bộ nhớ...")
        gc.collect()

# Chuyển thành Dataset
all_dataset_processed = Dataset.from_list(processed_all_data)
del processed_all_data

gc.collect()

# Sau khi đã xử lý xong, tách lại thành train/val/test theo số lượng ban đầu
n_train = len(train_dataset)
n_val = len(val_dataset)
n_test = len(test_dataset)

train_dataset_processed = all_dataset_processed.select(range(0, n_train))
val_dataset_processed = all_dataset_processed.select(range(n_train, n_train + n_val))
test_dataset_processed = all_dataset_processed.select(range(n_train + n_val, n_train + n_val + n_test))

Đã xử lý 100 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 200 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 200 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 300 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 300 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 400 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 400 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 500 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 500 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 600 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 600 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 700 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 700 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 800 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 800 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 900 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 900 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 1000 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 1000 mẫu huấn luyện. Giải phóng bộ nhớ...
Đã xử lý 1100 mẫu huấn luyện. Giải phóng bộ nhớ.

## 5. Training Arguments and Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="./whisper-vi-finetuned",
    per_device_train_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=1e-4,
    warmup_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
)

def compute_metrics(pred):
    from jiwer import wer
    pred_str = processor.batch_decode(pred.predictions, skip_special_tokens=True)
    label_str = processor.batch_decode(pred.label_ids, skip_special_tokens=True)
    return {"wer": wer(label_str, pred_str)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=val_dataset_processed,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

## 6. Start Training

In [ ]:
trainer.train()

## 7. Evaluate on Test Set

In [ ]:
test_results = trainer.evaluate(test_dataset_processed)
print(test_results)

In [ ]:
# Save the fine-tuned model and processor for later use
model.save_pretrained("./whisper-vi-finetuned")
processor.save_pretrained("./whisper-vi-finetuned")
print("Model and processor saved to ./whisper-vi-finetuned")

---
This notebook provides a basic pipeline for fine-tuning Whisper on Vietnamese ASR data. You can further customize preprocessing, augmentation, and hyperparameters as needed.